In [5]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East_Massachusetts_Jul_NT")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [6]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 1e3
    l = 1
    #average_excess_cost = 1
    tol = 1e-5

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [7]:
# getting the coefficients of the costs
coeffs_dict_Jul_NT = readall("../temp_files/coeffs_dict_Jul_NT.json")
coeffs_dict_Jul_NT = JSON.parse(coeffs_dict_Jul_NT)

xl = Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}}()

deg_grid = [4:8]
c_grid = .5:.5:3.
lamb_grid = [10. .^(-3:4)]

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_1 = coeffs_dict_Jul_NT["($(deg),$(c),$(lam),1)"]
            coeffs_2 = coeffs_dict_Jul_NT["($(deg),$(c),$(lam),2)"]
            coeffs_3 = coeffs_dict_Jul_NT["($(deg),$(c),$(lam),3)"]
            xl[(deg, c, lam, 1)] = MSA(coeffs_1)
            xl[(deg, c, lam, 2)] = MSA(coeffs_2)
            xl[(deg, c, lam, 3)] = MSA(coeffs_3)
        end
    end
end

In [8]:
lamb_grid

8-element Array{Float64,1}:
     0.001
     0.01 
     0.1  
     1.0  
    10.0  
   100.0  
  1000.0  
 10000.0  

In [9]:
outfile = open("../temp_files/uni-class_traffic_assignment_MSA_flows_Jul_NT.json", "w")

JSON.print(outfile, xl)

close(outfile)

In [10]:
xl

Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}} with 720 entries:
  (8,2.5,10.0,3)    => [7060.206164,7688.108581253005,0.0,9.789305747000004,146…
  (4,1.0,0.01,2)    => [3233.5744231119975,3367.0666924090015,3826.919128970999…
  (4,3.0,0.001,1)   => [3233.5744231119984,3368.3862930089986,3826.919128971,43…
  (8,2.0,1000.0,3)  => [3229.6241030359997,3376.0392737779985,3837.495660738002…
  (6,1.5,0.1,1)     => [3236.684309199999,3374.2065688459998,3830.4354545740007…
  (6,1.5,100.0,1)   => [3236.6843091999986,3376.332178825,3830.435454574,4328.4…
  (4,2.0,1000.0,3)  => [3240.6346292760013,3364.500683860003,3819.858922806999,…
  (5,2.5,10.0,1)    => [3240.6346292760018,3364.2072852409956,3819.858922806998…
  (7,0.5,1.0,3)     => [3223.9818993900008,3384.1591315239966,3848.204305405001…
  (5,0.5,100.0,2)   => [3229.624103036,3380.290973941997,3837.4956607380013,432…
  (7,1.5,1.0,3)     => [3236.6843091999995,3374.2060819569974,3830.435454573999…
  (4,0.5,0.01,1)    => [3236.684309199

In [11]:
xl[(8,2.5,0.1,2)]

24-element Array{Float64,1}:
 10730.3     
 11888.8     
 16822.4     
 16301.6     
     0.0     
     0.0     
 12008.2     
 13888.6     
 11524.7     
  6269.04    
     0.0     
  4273.32    
     0.256599
     0.008077
     0.0     
     0.002465
  7633.22    
  9664.25    
  5960.48    
     0.0     
  6441.24    
 10911.7     
  6398.01    
  6043.54    